# Segment Anything Model as Inference

In [ ]:
import os
import shutil
import csv

import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

from skimage.filters import sobel, rank
from skimage.segmentation import watershed, felzenszwalb
from scipy import ndimage as ndi
from skimage.morphology import disk

from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
import sklearn.metrics

In [ ]:
# Define the path to the data directory and load the train labels
DATA_PATH = "./../../data/"
data_dir = Path(DATA_PATH)
labels_train = pd.read_csv(data_dir / "Y_train.csv", index_col=0).T


In [30]:
# We create a file containing only labeled X_train images to compute metrics using SAM for inference.
path_to_X_train = f'{DATA_PATH}X_train/'
path_to_X_train_ground_truth = f'{DATA_PATH}X_train_ground_truth'
os.mkdir(path_to_X_train_ground_truth)

# Copy the labeled images to the new folder
for i in range(201):
    image_path = os.path.join(path_to_X_train, str(i) + '.png')
    ground_truth_path = os.path.join(
        path_to_X_train_ground_truth, str(i) + '.png')
    shutil.copyfile(image_path, ground_truth_path)

# Set the directory for inferenced images
inference_dir = f'{DATA_PATH}X_train_ground_truth/'

# Create the predictions directory folder if it does not exist
predictions_dir = f'{DATA_PATH}predictions/inference_sam/'
os.makedirs(predictions_dir, exist_ok=True)


In [8]:
# Load the pre-trained Vision Transformer model using the SAM library
sam = sam_model_registry["vit_l"](
    checkpoint="./checkpoints_sam/sam_vit_l_0b3195.pth")


In [36]:
# Loop over each image in the directory
for filename in os.listdir(inference_dir):
    # Read the image and convert it to RGB
    image = cv2.imread(os.path.join(inference_dir, filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Generate the mask using the SamAutomaticMaskGenerator object
    mask_generator = SamAutomaticMaskGenerator(sam)
    masks = mask_generator.generate(image)

    # Remove the file extension for naming the files when saving outputs
    filename_without_extension = os.path.splitext(filename)[0]

    # Save predicted outputs to CSV file
    with open(f'{predictions_dir}{filename_without_extension}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['segmentation', 'area', 'bbox', 'predicted_iou',
                        'point_coords', 'stability_score', 'crop_box'])
        for mask in masks:
            writer.writerow([mask['segmentation'], mask['area'], mask['bbox'], mask['predicted_iou'],
                            mask['point_coords'], mask['stability_score'], mask['crop_box']])


# ajouter viz entre inférence et ground truth 

In [ ]:
# Plot the image and mask
plt.figure(figsize=(7, 7))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()


ajouter métriques